# Face Recognition
### 1) Import libraries
### 2) Import video
### 3) Convert to b/w
### 4) Extract frames
### 5) Mark the region of face
### 6) Create Custom Dataset for each person
### 7) Create Train and test loaders
### 8) Create CNN Algorithm
### 9) Test and improve accuracy
### 9) Identify real time image

After we create the b/w images, then we will pass the images through an algorithm which will give the position of face in the image, then we will crop(resize) the image to that shape and we will fix a certain shape so as to pad these images and make every image of the same size. These images will be the input to Siamese Network.


In [31]:
import cv2
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import OrderedDict
from tqdm import tqdm
import glob

import random
import matplotlib.pyplot as plt
from PIL import Image
from itertools import combinations 

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import torchvision
import torchvision.transforms as transforms

In [3]:
person_name = OrderedDict()
person_name['Venkatesh'] = 1
person_name['Vivek'] = 2
person_name['Prince'] = 3
person_name['Abhay'] = 4

## Importing video and extracting frames

In [4]:
# video = cv2.VideoCapture('Sample.mp4')
# fps = video.get(cv2.CAP_PROP_FPS)
# print(fps)

# minutes = 0
# seconds = 5
# total_frames = video.get(cv2.CAP_PROP_FRAME_COUNT)
# # seconds = total_frames//fps
# # minutes = seconds//60
# # seconds = seconds % 60
# # We can use os.chdir(path) to change the working directory
# arr = np.array([])
# width, height =  video.get(cv2.CAP_PROP_FRAME_WIDTH), video.get(cv2.CAP_PROP_FRAME_HEIGHT)
# df = pd.DataFrame(columns = np.arange(int(width)*int(height)))
# for i in range(0,int(total_frames),10):
# #     frame_id = int(fps*(minutes*60 + seconds))
#     frame_id = i
# #     print('frame id =',frame_id)
#     video.set(cv2.CAP_PROP_POS_FRAMES, frame_id)
#     ret, frame = video.read()
# #     gray_image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#     gray_image = frame
# #     print(type(frame))
# #     cv2.imshow('frame', frame); cv2.waitKey(0)
# #     cv2.imwrite(f'{frame_id}.png', frame)
#     img = np.asarray(gray_image)
# #     imgs = pd.DataFrame(img.reshape(1,-1))
# #     pd.concat([df,imgs])
# #     df.loc[i] = imgs
#     path = 'C:\\Users\\dhano\\Jupyter Notebooks\\Face Recognition\\Venkatesh'
#     cv2.imwrite(os.path.join(path , f'{frame_id}.jpg'), gray_image)
# print("done")

## Crop the image till its face

In [5]:
# import cv2
# imagePath = 'sample.jpg'
# img = cv2.imread(imagePath) # This is already a numpy array
# gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# face_classifier = cv2.CascadeClassifier(
#     cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
# face = face_classifier.detectMultiScale(
#     gray_image, scaleFactor=1.1, minNeighbors=5, minSize=(40, 40))
# for (x, y, w, h) in face:
#     cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 4)
# im_crop = gray_image[y:y+h, x:x+w] # Crop the image so that only face remains
# # im_crop = cv2.cvtColor(im_crop, cv2.COLOR_BGR2GRAY)

# # img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
# im_crop = cv2.cvtColor(im_crop, cv2.COLOR_BGR2RGB)
# import matplotlib.pyplot as plt
# plt.figure(figsize=(20,10))
# plt.imshow(im_crop)
# plt.axis('off')
# im_crop.shape

##  Function to extract faces from video and save them in a different folder

In [49]:
# Function to extract images from video, and save just the faces from all the images in a seperate folder
def video_to_faces(video_path, label):
    video = cv2.VideoCapture(video_path)
    if not video.isOpened():
        print(f"Error: Unable to open video file at {video_path}")
        return
    total_frames = video.get(cv2.CAP_PROP_FRAME_COUNT)
    # We can use os.chdir(path) to change the working directory
#     arr = np.array([])
    width, height =  video.get(cv2.CAP_PROP_FRAME_WIDTH), video.get(cv2.CAP_PROP_FRAME_HEIGHT)
    df = pd.DataFrame(columns = np.arange(int(width)*int(height)))
    face_classifier = cv2.CascadeClassifier(
        cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    max_h, max_w = 0,0
#     path = 'C:\\Users\\dhano\\Jupyter Notebooks\\Face Recognition\\' + str(label)
    path = video_path.replace('Videos', 'Images')
    path = path.replace('.mp4', '')
#     if os.path.exists(path):
#         continue
#     else:
    #     print(path)
    os.makedirs(path, exist_ok=True)  # Ensure the directory exists or create it
    for i in range(0,int(total_frames),10):
        frame_id = i
        file_path = os.path.join(path , f'{frame_id}.jpg')
        if os.path.exists(file_path):
            continue
        else:
            video.set(cv2.CAP_PROP_POS_FRAMES, frame_id)
            ret, frame = video.read()
            if not ret:
                print(f"Error reading frame {frame_id}")
                continue
            gray_image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            face = face_classifier.detectMultiScale(
                gray_image, scaleFactor=1.1, minNeighbors=9, minSize=(40, 40))
            x, y, w, h = 0,0,0,0
    #         print(face)
            if len(face) == 0: ## If no face is detected then it will skip this frame
                continue
            for (x, y, w, h) in face:
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 4)
                x, y, w, h = x, y, w, h
            im_crop = gray_image[y:y+h, x:x+w]
    #         im_crop = cv2.cvtColor(im_crop, cv2.COLOR_BGR2RGB)
            im_crop = cv2.cvtColor(im_crop, cv2.COLOR_GRAY2RGB)
            im_h, im_w, _ = im_crop.shape
            max_h, max_w = max(im_h, max_h), max(im_w, max_w) # Find the max shape of image
    #         import matplotlib.pyplot as plt
    #         plt.figure(figsize=(10,10))
    #         plt.imshow(im_crop)
    #         plt.axis('off')
    #         path = 'C:\\Users\\dhano\\Jupyter Notebooks\\Face Recognition\\' + str(label)
            cv2.imwrite(file_path, im_crop)
    print("done")
    return max_h, max_w 
# max_h, max_w = video_to_faces('Abhay Singh.mp4', "Abhay singh")
# max_h, max_w = video_to_faces('C:\\Users\\dhano\\Jupyter Notebooks\\Face Recognition\\Dataset\\Videos\\Abhay Singh.mp4', "Abhay Singh")
# print(max_h, max_w)
plt.close()

## Code to convert all the videos to face images and store them in seperate folders according to their names

In [50]:
## Just Replace this path with the path to your video directory
video_dir = "C:\\Users\\dhano\\Jupyter Notebooks\\Face Recognition\\Dataset\\Videos"
## Code to be used later to get a list of sub-folders in a folder
## folders = [x[0] for x in os.walk(video_dir)]
videos_list = [f for f in os.listdir(video_dir) if f.endswith(('.mp4'))]
max_list = []
for video in tqdm(videos_list):
#     print("Working on", video) 
    video_path = video_dir + "\\" + str(video)
    label = video.replace('.mp4','')
#     os.mkdir(label)
    max_h, max_w = video_to_faces(video_path, label)
#     print(max_h, max_w)
    max_list.append([max_h,max_w])
print("Done")

Shrinath Mishra.mp4:   3%|██                                                            | 1/30 [00:15<07:27, 15.45s/it]

done


Shrinath Mishra.mp4:   7%|████▏                                                         | 2/30 [02:00<31:51, 68.26s/it]

done


Shrinath Mishra.mp4:  10%|██████▏                                                       | 3/30 [03:48<38:50, 86.30s/it]

...

done


Shrinath Mishra.mp4:  97%|██████████████████████████████████████████████████████████▉  | 29/30 [32:04<00:47, 47.84s/it]

done


Shrinath Mishra.mp4: 100%|█████████████████████████████████████████████████████████████| 30/30 [32:27<00:00, 64.92s/it]

done
Done


### Function to pad the image to a certain size

In [41]:
# max_list = np.array(max_list).astype(int)
max_h, max_w = (np.max(max_list,axis = 0))
print(max_h, max_w)
def pad(img, max_h, max_w):
    im_h, im_w, _ = img.shape
    left = (max_w - im_w)//2
    right = max_w - im_w - left
    top = (max_h - im_h)//2
    bottom = max_h - im_h - top 
#     print(max_h, max_w, im_h, im_w)
    img_pad = np.pad(img, pad_width=((top,bottom), (left, right), (0, 0)))
    return img_pad
# print(max_h, max_w)
# img = cv2.imread("C:\\Users\\dhano\\Jupyter Notebooks\\Face Recognition\\Venkatesh\\330.jpg")
# print(img.shape)
# img_pad = pad(img, max_h, max_w)
# plt.imshow(img)
# plt.imshow(img_pad)

841 841


## Function to store the padded images in seperate folders

In [44]:
# label, dataframe, label_dict
# image_data = pd.DataFrame(columns = np.arange(int(max_h)*int(max_w)+1))
def pad_and_store(images_path, max_h, max_w):
    '''
    images_path = Path of the folder inside which there are folders of each person's name
                  containing images of their faces
    max_h, max_w = Max height and width of the images of faces, we will pad all other images
                    to make them of this shape
    '''
    # List of folders inside the Images folder
    folders = [x[0] for x in os.walk(images_path)]
    for folder in tqdm(folders):
        # Get a list of all image files in the folder
        folder_path = os.path.join(images_path,folder)
        image_files = [f for f in os.listdir(folder_path) if f.endswith(('.jpg'))]
        pad_image_dir = folder_path.replace("Images", "Images_padded")
        os.makedirs(pad_image_dir, exist_ok=True)
        # Loop through the image files and open them
        for image_file in image_files:
            # Construct the full file path
            file_path = os.path.join(folder_path, image_file)
            save_path = os.path.join(pad_image_dir, image_file)
            if os.path.exists(save_path):
                continue
            else:
                image = cv2.imread(file_path)
                img_pad = pad(image, max_h, max_w)
                cv2.imwrite(save_path, img_pad)
#             row = list(od[label])
#             img_pad_flat = img_pad.reshape(1,-1)
#             row.append(img_pad_flat)
#             image_data.append(row)
    print("Done")
    return 
image_dir = video_dir.replace("Videos", "Images")
image_data = pad_and_store(image_dir, max_h, max_w)

100%|█████████████████████████████████████████████████████████████████████████████████| 31/31 [00:00<00:00, 424.87it/s]

Done


## Generate Random Triplets

In [ ]:
def generate_random_triplets(images_path):
    '''
    We will choose the number of triplets based on the number of images in each folder.
    Lets say we will choose at max 8 pairs of anchor and positive images from a folder.
    If a folder has less than 5 images then we will skip it.
    We will calculate the number of images in a folder then by using nC2 we will calculate number of 
    pairs we can choose.
    '''
    folders = [x[0] for x in os.walk(images_path)]
    for folder in folders:
        path = os.path.join(images_path)
        image_files = [f for f in os.listdir(folder_path) if f.endswith(('.jpg'))]
        if len(image_files) < 5:
            continue
        

image_dir = video_dir.replace("Videos", "Images")
generate_random_triplets(image_dir)

## Triplet loss function

In [ ]:
def triplet_loss(anchor, positive, negative, margin=0.2):
    pos_dist = (anchor - positive).pow(2).sum(-1) #.pow(.5)
    neg_dist = (anchor - negative).pow(2).sum(-1) #.pow(.5)
    loss = F.relu(pos_dist - neg_dist + margin)
    return loss.mean()

# Code to Open Webcam, draw a rectangle around face and label it


In [29]:
# access video from the webcam
video_capture = cv2.VideoCapture(0)
face_classifier = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
def detect_bounding_box(vid):
    gray_image = cv2.cvtColor(vid, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray_image, 1.1, 5, minSize=(40, 40))
    x, y, w, h = 0,0,0,0
    for (x, y, w, h) in faces:
        cv2.rectangle(vid, (x, y), (x + w, y + h), (0, 255, 0), 4)
        x, y, w, h = x, y, w, h
    return faces, x, y
while True:
    result, video_frame = video_capture.read()  # read frames from the video
    if result is False:
        break  # terminate the loop if the frame is not read successfully
    faces, x, y = detect_bounding_box(
        video_frame
    )  # apply the function we created to the video frame
    title = 'Face'
    cv2.putText(video_frame,title , (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (30,255,30), 2)
    cv2.imshow(
        "My Face Detection Project", video_frame
    )
    # display the processed frame in a window named "My Face Detection Project"
    # add label to the face
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
video_capture.release()
cv2.destroyAllWindows()